In [1]:
## Importando as libs necessárias
from google.colab import files
import os, io, time

# Criando o diretório inicial: /content/
os.chdir('/content/')
try:
  os.mkdir('Imagens', )
except:
  print('A pasta já existe.')
os.chdir('./Imagens')
os.listdir()
uploaded_images = files.upload()

ModuleNotFoundError: No module named 'google'

In [5]:
## Reduzir as imagens
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog

hog_features = []
for filename in uploaded_images.keys():
    image = imread(filename)
    image_resized = resize(image, (128, 128))
    fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(20, 20),
                        cells_per_block=(2, 2), visualize=True, channel_axis=-1)
    hog_features.append(fd)

# Substituindo as raças de cachorros e gatos por 'dog' e 'cat'
classe = ['dog' if 'american_bulldog' in file else 'cat' if 'Bengal' in file
           else file for file in uploaded_images.keys()]

import pandas as pd
## Adicionar as colunas com os valores do hog + classe
data = {"hog_features": hog_features, "classe": classe}
df = pd.DataFrame(data)

# Transformando cada valor do hog_feature em um atributo (coluna)
df2 = pd.DataFrame(df['hog_features'].tolist())
df2.columns = df2.columns.map(lambda x: f'hog_feature_{x+1}')
df = pd.concat([df.drop('hog_features', axis=1), df2], axis=1)

# Mover a coluna 'classe' para o final
df['classe'] = df.pop('classe')

# Salvando csv resultante
df.to_csv('HOG_128_20x20.csv', sep=',', index=False)